In [18]:
import os
from dotenv import load_dotenv
import numpy as np
from geopy.distance import geodesic
import pandas as pd
import geopandas as gpd
import time
from pathlib import Path
import googlemaps




from geopy.geocoders import Nominatim


PROJECT_ROOT = Path().absolute().parent
DATA_DIR = PROJECT_ROOT / 'data'
RAW_DATA_DIR = DATA_DIR / 'raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'
TEMP_DATA_DIR = DATA_DIR / 'temp'

#load in data (concatenate 2 sets immediately), structure is the same, kick-out  leading index column. 
roadpoints=pd.concat([pd.read_csv(TEMP_DATA_DIR / 'aisa_roadpoints.csv'), pd.read_csv(TEMP_DATA_DIR / 'dwanwana_dokolo_roadpoints.csv')], ignore_index=True).drop(columns=['Unnamed: 0'])


In [7]:
print(roadpoints.station_name.value_counts(dropna=False))
print(roadpoints.head()) #seems about ok.

station_name
Aisa FM        70
Dwanwana FM    70
Dokolo FM      70
Name: count, dtype: int64
   grid_id station_name  buffer_km  est_population_2020  centroid_lat  \
0    24524      Aisa FM       25.0           425.846130      1.324168   
1    23364      Aisa FM       25.0           661.170654      1.595578   
2    18590      Aisa FM       25.0           517.236816      1.469014   
3    22674      Aisa FM       25.0           730.694519      1.505130   
4    20788      Aisa FM       25.0            92.748978      1.360421   

   centroid_lon                                 centroid_maps_link  \
0     33.919442  https://www.google.com/maps?q=1.32416848783960...   
1     33.856628  https://www.google.com/maps?q=1.59557827040676...   
2     33.604888  https://www.google.com/maps?q=1.46901386082312...   
3     33.820636  https://www.google.com/maps?q=1.50512989025721...   
4     33.721712  https://www.google.com/maps?q=1.36042052665992...   

  cluster_type  nearest_road_lat  nearest_road_

In [ ]:
def get_location_details(lat, lon):
    """
    Get administrative areas/adresses using OpenStreetMap/Nominatim
    """
    geolocator = Nominatim(user_agent="my_app")
    
    try:
        # Make the reverse geocoding request
        location = geolocator.reverse((lat, lon), exactly_one=True)
        if not location:
            return None
            
        # Extract address components
        address = location.raw['address']
        
        location_data = {'location_full': location,
            'village': address.get('village'),
             'district': address.get('state') or address.get('district'),
             'region': address.get('region'),
             'country': address.get('country')
         }
        
        return location
        
    except Exception as e:
        print(f"Error: {e}")
        return None



In [ ]:
#apply to dataframe
results=[]
for index, row in roadpoints.iterrows():
    location_data = get_location_details(row['nearest_road_lat'], row['nearest_road_lon'])
    if location_data:
        location_data['grid_id'] = row['grid_id'] #parse the grid id from the original df for indexing/merging. 
        print(location_data)
        results.append(location_data)
        time.sleep(0.01)  

Error: Must be a coordinate pair or Point
{'location_full': Location(Doyoro 'a', Ngora, Eastern Region, Uganda, (1.5998745, 33.859485, 0.0)), 'village': "Doyoro 'a'", 'district': 'Ngora', 'region': 'Eastern Region', 'country': 'Uganda', 'grid_id': 23364}
{'location_full': Location(Akisim, Serere, Eastern Region, Uganda, (1.4691756913236675, 33.604843895590506, 0.0)), 'village': 'Akisim', 'district': 'Serere', 'region': 'Eastern Region', 'country': 'Uganda', 'grid_id': 18590}
{'location_full': Location(Kalengo, Ngora, Eastern Region, Uganda, (1.5054595385927452, 33.82133801981945, 0.0)), 'village': 'Kalengo', 'district': 'Ngora', 'region': 'Eastern Region', 'country': 'Uganda', 'grid_id': 22674}
{'location_full': Location(Oceeren, Ngora, Eastern Region, Uganda, (1.3628304684396821, 33.71733325508234, 0.0)), 'village': 'Oceeren', 'district': 'Ngora', 'region': 'Eastern Region', 'country': 'Uganda', 'grid_id': 20788}
{'location_full': Location(Oceeren, Ngora, Eastern Region, Uganda, (1.36

In [83]:
#throw in dataframe
results_df=pd.DataFrame(results)
#how many districts
print(results_df.district.value_counts(dropna=False))
#temp outfile save
results_df.to_csv(TEMP_DATA_DIR/'reversegeocod.csv')

#try to extract the coordinate of the nearest village that I get through nomatim/open streetmap.  Seems that some reverse geocoded villages are far away given lat >2.xxxx

test=results_df['location_full'].str.extract(r'\(([-\d.]+),\s*([-\d.]+)').astype(float) #thanks gpt for regexing this. 

test# print(results_df.location_full[0])



district
Dokolo           56
Kaberamaido      41
Kalaki           28
Kumi             26
Ngora            22
Pallisa          14
Lira District     8
Alebtong          6
Serere            5
Soroti            2
Kwania            1
Name: count, dtype: int64


,0,1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
204,NaN,NaN
205,NaN,NaN
206,NaN,NaN
207,NaN,NaN


In [85]:
geolocator = Nominatim(user_agent="my_app")

test=geolocator.reverse(((1.469210,33.604877)), exactly_one=False)
test.raw


AttributeError: 'list' object has no attribute 'raw'

In [69]:
results_df.location_full[0]

Location(Doyoro 'a', Ngora, Eastern Region, Uganda, (1.5998745, 33.859485, 0.0))

In [82]:
results_df.columns
results_df

,location_full,village,district,region,country,grid_id
0,"(Doyoro 'a', Ngora, Eastern Region, Uganda, (1...",Doyoro 'a',Ngora,Eastern Region,Uganda,23364
1,"(Akisim, Serere, Eastern Region, Uganda, (1.46...",Akisim,Serere,Eastern Region,Uganda,18590
2,"(Kalengo, Ngora, Eastern Region, Uganda, (1.50...",Kalengo,Ngora,Eastern Region,Uganda,22674
3,"(Oceeren, Ngora, Eastern Region, Uganda, (1.36...",Oceeren,Ngora,Eastern Region,Uganda,20788
4,"(Oceeren, Ngora, Eastern Region, Uganda, (1.36...",Oceeren,Ngora,Eastern Region,Uganda,20788
...,...,...,...,...,...,...
204,"(Agora, Kalaki, Eastern Region, Uganda, (1.882...",Agora,Kalaki,Eastern Region,Uganda,14386
205,"(Okeratok A, Kaberamaido, Eastern Region, Ugan...",Okeratok A,Kaberamaido,Eastern Region,Uganda,10121
206,"(Oselel, Kalaki, Eastern Region, Uganda, (2.02...",Oselel,Kalaki,Eastern Region,Uganda,13552
207,"(Kaberamaido, Eastern Region, Uganda, (1.78618...",None,Kaberamaido,Eastern Region,Uganda,10465
